In [1]:
from modules.data import data_loader, data_splitter
from modules.models import arima, lag_llama

In [2]:
import pandas as pd

In [3]:
prediction_length = 10

In [4]:
data = data_loader.get_simle_data()
train, test = data_splitter.split_data(data, prediction_length)

[*********************100%%**********************]  1 of 1 completed


In [9]:
train.dtypes

y    float64
dtype: object

In [6]:
arima_model = arima.get_autoarima(train)

In [7]:
autoarima_predictions = arima.autoarima_predictions(arima_model, prediction_length)

In [8]:
lag_llama_predictions, tss = lag_llama.get_lam_llama_forecast(train, prediction_length)

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float